# 策略梯度
最基础的策略梯度算法就是REINFORCE算法，又称Monte-Carlo Gradient算法。我们策略优化的目标如下：
$$
J_\theta = \Psi_\pi \nabla_\theta \log \pi_\theta(a_t|s_t)
$$
其中$\Psi_\pi$在REINFORCE算法中表示衰减的回报。也可以用优势来估计，也就是我们熟知的A3C算法。
我们介绍一下策略梯度中最简单的也是最经典的一个算法 REINFORCE。REINFORCE 用的是回合更新的方式，它在代码上的处理上是先获取每个步骤的奖励，然后计算每个步骤的未来总奖励 $G_t$，将每个 $G_t$ 代入
$$
\nabla\bar{R}_\theta\approx\frac{1}{N}\sum_{n = 1}^{N}\sum_{t = 1}^{T_n}G_t^n\nabla\log\pi_\theta(a_t^n|s_t^n)
$$ (4.21)

优化每一个动作的输出。所以我们在编写代码时会设计一个函数，这个函数的输入是每个步骤获取的奖励，输出是每一个步骤的未来总奖励。因为未来总奖励可写为
$$
\begin{align*}
G_t&=\sum_{k = t + 1}^{T}\gamma^{k - t - 1}r_k\\
&=r_{t + 1}+\gamma G_{t + 1}
\end{align*}
$$ (4.22)

即上一个步骤和下一个步骤的未来总奖励的关系如式 (4.22) 所示，所以在代码的计算上，我们是从后往前推，一步一步地往前推，先算 $G_T$，然后往前推，一直算到 $G_1$。

如图 4.14 所示，REINFORCE 的伪代码主要看最后 4 行，先产生一个回合的数据，比如
$$
(s_1,a_1,G_1),(s_2,a_2,G_2),\cdots,(s_T,a_T,G_T)
$$

然后针对每个动作计算梯度 $\nabla\log\pi(a_t|s_t,\theta)$。在代码上计算时，我们要获取神经网络的输出。神经网络会输出每个动作对应的概率值（比如 0.2、0.5、0.3），然后我们还可以获取实际的动作 $a_t$，把动作转成独热（one - hot）向量（比如 $[0,1,0]$）与 $\log[0.2,0.5,0.3]$ 相乘就可以得到 $\log\pi(a_t|s_t,\theta)$。 

## 策略函数设计
策略梯度算法是直接对策略函数进行梯度计算，那么策略函数的设计就显得关键了。一般有两种设计方式，一种是softmax函数，另外一个是高斯分布 $\mathbb{N}(\phi(\mathbb{s})^{\pi}\theta,\sigma^2)$，前者用于离散动作空间，后者多用于连续动作空间。

softmax函数可以表示为：

$$
\pi(a|\mathbb{s}) = \frac{e^{\phi(\mathbb{s})^{T_\theta}}}{\sum_{b} e^{\phi(s,b)^{T_\theta}}}
$$

其中，$\phi(s)^{\pi}\theta$为策略函数，$\theta$为策略函数的参数，$\sigma^2$为噪声方差。对应的梯度为
$$
\nabla_\theta \log \pi_\theta(s, a)=\phi(s, a)-\mathbb{E}_{\pi_\theta}[\phi(s,a)]
$$
高斯分布对应的梯度为：
$$
\nabla_\theta \log \pi_\theta(s, a)=\frac{\left(a-\phi(s)^T \theta\right) \phi(s)}{\sigma^2}
$$
但是对于一些特殊的情况，例如在本次演示中动作维度=2且为离散空间，这个时候可以用伯努利分布来实现，这种方式其实是不推荐的，这里给大家做演示也是为了展现一些特殊情况，启发大家一些思考，例如Bernoulli，Binomial，Gaussian分布之间的关系。简单说来，Binomial分布，$n=1$ 时就是Bernoulli分布，$n\rightarrow \infty$ 时就是Gaussian分布.

## 模型设计
前面讲到，尽管本次演示是离散空间，但是由于动作维度等于2，此时就可以用特殊的高斯分布来表示策略函数，即伯努利分布。伯努利的分布实际上是用一个概率作为输入，然后从中采样动作，伯努利采样出来的动作只可能是0或1，就像投掷出硬币的正反面。在这种情况下，我们的策略模型就需要在MLP的基础上，将状态作为输入，将动作作为倒数第二层输出，并在最后一层增加激活函数来输出对应动作的概率。不清楚激活函数作用的同学可以再看一遍深度学习相关的知识，简单来说其作用就是增加神经网络的非线性。既然需要输出对应动作的概率，那么输出的值需要处于0-1之间，此时sigmoid函数刚好满足我们的需求，实现代码参考如下。

### 导入相关的包

In [210]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical, Bernoulli
from torch.autograd import Variable
import numpy as np
import argparse
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import gym
import sys
import os
#from utils import *
#from utils import smooth, plot_rewards

In [ ]:
# 定义策略网络pi_\theta
class PGNet(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=128):
        """Args:
            初始化Q网络，为全连接网络
            input_dim: 输入的特征数即环境的状态维度
            output_dim: 输出的动作维度
        """
        super(PGNet, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            #nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            #nn.ReLU(),
            nn.Linear(hidden_dim, output_dim),
            #nn.sigmoid()
        )
        #self.fc1 = nn.Linear(input_dim, hidden_dim)
        #self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        #self.fc3 = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        #x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        #x = torch.sigmoid(self.fc3(x))
        return torch.softmax(self.net(x))
    
    def save_checkpoint(self, checkpoint_file):
        torch.save(self.state_dict(), checkpoint_file, _use_new_zipfile_serialization=False)
 
    def load_checkpoint(self, checkpoint_file):
        self.load_state_dict(torch.load(checkpoint_file))


## 更新函数设计
对于梯度我们需要拆开成两个部分$\Psi_\pi$和$\nabla_\theta \log \pi_\theta (a_t | s_t)$分开计算，首先看值函数部分$\Psi_\pi$，在REINFORCE算法中值函数是从当前时刻开始的衰减回报，如下：
$$
G \leftarrow \sum_{k = t + 1}^{T} \gamma^{k - 1}r_k
$$
这个实际用代码来实现的时候可能有点绕，我们可以倒过来看，在同一回合下，我们的终止时刻是$T$，那么对应的回报$G_T = \gamma^{T - 1}r_T$，而对应的$G_{T - 1} = \gamma^{T - 2}r_{T - 1} + \gamma^{T - 1}r_T$，在这里代码中我们使用了一个动态规划的技巧，如下：

```python
running_add = running_add * self.gamma + reward_pool[i]  # running_add初始值为0
```

这个公式也是倒过来循环的，第一次的值等于：
$$
running\_add = r_T
$$
第二次的值则等于：
$$
running\_add = r_T * \gamma + r_{T - 1}
$$
第三次的值等于：
$$
running\_add = (r_T * \gamma + r_{T - 1}) * \gamma + r_{T - 2} = r_T * \gamma^2 + r_{T - 1} * \gamma + r_{T - 2}
$$

In [212]:
class PolicyGradient:
    #def __init__(self, model, memory, cfg):
    def __init__(self, model, cfg):
        self.gamma  = cfg.gamma # 折扣因子
        self.device = cfg.device
        #self.memory = memory
        self.log_probs = []
        self.rewards = []
        self.checkpoint_dir = cfg.ckpt_dir
        if cfg.if_load_ckpt:
            self.model = model.load_checkpoint(self.checkpoint_dir + sorted(os.listdir(self.checkpoint_dir))[-1])
        else:
            self.policy_net = model.to(self.device)
        self.optimizer = torch.optim.Adam(self.policy_net.parameters(), lr=cfg.lr)
        self.n_actions = cfg.n_actions
        self.n_states = cfg.n_states
        
    

    def sample_action(self, state):
        """从当前状态采样动作
        Args:
            state: 状态"""
        #state = torch.from_numpy(state).float() # 将numpy数组转换为PyTorch张量
        state = F.one_hot(torch.tensor(state), num_classes=self.n_states).float().to(self.device)
        #state = Variable(state) # 包装张量，使其能够记录操作并支持梯度计算
        #print(state)
        probs = self.policy_net(state) # 输入状态，输出动作概率
        m = Categorical(probs) # 建立一个分布
        action = m.sample() # 采样一个动作
        self.log_probs.append(m.log_prob(action)) # 保存log概率用于梯度计算

        #action = action.data.numpy().astype(int)[0] # 转化为标量

        return action.item()
    
    def predict_action(self, state):
        """从当前状态预测下一步的动作
        """
        #state = torch.from_numpy(state).float()
        state = F.one_hot(torch.tensor(state, dtype=int), num_classes=self.n_states).int()
        state = Variable(state)
        probs = self.policy_net(state)
        m = Categorical(probs)
        action = m.sample()
        action = action.data.numpy().astype(int)[0]
        #?????和采样动作有区别吗？
        return action
    
    def save_models(self, episode):
        self.policy_net.save_checkpoint(self.checkpoint_dir + 'Reinforce_policy_{}.pth'.format(episode))
        print('Saved the policy network successfully!')

    def load_models(self, episode):
        self.policy_net.load_checkpoint(self.checkpoint_dir + 'Reinforce_policy_{}.pth'.format(episode))
        print('Loaded the policy network successfully!')

    def update(self):
        """train_step()
        """
        #state_pool, action_pool, reward_pool = self.memory.sample()
        #state_pool, action_pool, reward_pool = list(state_pool), list(action_pool), list(reward_pool)
        
        reward_pool = []
        
        # 折扣奖励
        running_add = 0
        for r in reversed(self.rewards):
            running_add = running_add * self.gamma + r
            reward_pool.insert(0, running_add)
        
        # 归一化
        rewards_normed = torch.tensor(reward_pool)
        rewards_normed = (rewards_normed - rewards_normed.mean()) / (rewards_normed.std() + 1e-7)
        '''reward_mean = np.mean(reward_pool)
        reward_std = np.std(reward_pool)
        for i in range(len(reward_pool)):
            reward_pool[i] = (reward_pool[i] - reward_mean) / (reward_std + 1e-7)
        '''

        policy_loss = []
        for log_prob, reward in zip(self.log_probs, rewards_normed):
            policy_loss.append(-log_prob * reward)
        
        # 梯度下降
        self.optimizer.zero_grad()
        '''
        for i in range(len(self.rewards)):
            #state = state_pool[i]
            #action = Variable(torch.FloatTensor([action_pool[i]]))
            reward = reward_pool[i]
            #state = Variable(torch.from_numpy(state).float())
            #probs = self.policy_net(state)
            #m = Bernoulli(probs)
            #loss = -m.log_prob(action) * reward #
            loss = -self.log_probs[i] * reward
            loss.backward()
        '''
        loss = torch.stack(policy_loss).sum()
        loss.backward()
        self.optimizer.step()
        #self.memory.clear()

        self.log_probs = []
        self.rewards = []

        return loss.item()

## 训练环境配置


### 训练测试函数

In [ ]:
def train(cfg, env, agent):
    '''
    - cfg: 配置参数
    - env: 环境
    - agent: 算法
    '''
    print("开始训练！")
    print(f'环境：{cfg.env_name}, 算法：{cfg.algo_name}, 设备：{cfg.device}')
    rewards = [] # 记录奖励
    losses = []
    for i_ep in range(cfg.train_eps):
        ep_reward = 0 # 一轮episode的reward
        state = env.reset() # 重置环境, 重新开一局（即开始新的一个episode）
        #action = agent.sample(state)
        while True:
            #print(state)
            action = agent.sample_action(state) # 根据算法采样一个动作
            #print(f"状态:{state}, 动作:{action}")
            next_state, reward, done, _ = env.step(action) # 与环境进行一个交互
            #print(next_state, done)
            #next_action = agent.sample_action(next_state)
            #agent.update(state, action, reward, next_state, next_action, done) # Sarsa算法更新
            #agent.update()
            #state = next_state
            #action = next_action
            agent.rewards.append(reward)
            ep_reward += reward
            state = next_state
            #ep_reward += reward
            if done:
                print("一次游戏结束.")
                break
        
        # 训练并记录数据
        loss = agent.update()
        losses.append(loss)
        rewards.append(ep_reward)

        if (i_ep+1) % 10 == 0: # 每50个回合打印一次信息
            print(f"回合：{i_ep+1}/{cfg.train_eps}，奖励：{np.mean(rewards):.1f}，损失：{loss:.3f}")

        if (i_ep+1) % 50 == 0:
            agent.save_models(i_ep+1)
    print("完成训练！")
    return {"rewards": rewards}

def test(cfg, env, agent):
    print("开始测试！")
    print(f"环境：{cfg.env_name}, 算法：{cfg.algo_name}, 设备：{cfg.device}")

    rewards = [] # 记录所有回合的episode奖励
    for i_ep in range(cfg.test_eps):
        ep_reward = 0 # 一轮episode的reward
        state = env.reset()
        while True:
            #print(i_ep)
            action = agent.predict_action(state) # 根据算法选择一个动作
            next_state, reward, done, _ = env.step(action) # 与环境进行一个交互
            state = next_state # 更新状态
            ep_reward += reward
            if done:
                break
        rewards.append(ep_reward)
        print(f"回合：{i_ep+1}/{cfg.test_eps}，奖励：{ep_reward:.1f}")
    print("完成测试！")
    return {"rewards": rewards}

### 环境和智能体配置

In [214]:
def env_agent_config(cfg):
    """创建环境和智能体
    """
    env = gym.make(cfg.env_name)
    #env = CliffWalkingWrapper(env)
    n_states = env.observation_space.n
    n_actions = env.action_space.n
    cfg.input_dim = n_states
    cfg.output_dim = n_actions
    cfg.n_states = n_states
    cfg.n_actions = n_actions
    agent = PolicyGradient(
        model=PGNet(input_dim=cfg.input_dim, 
                    output_dim=cfg.output_dim,
                    hidden_dim=cfg.hidden_dim), 
        cfg=cfg)
    return env, agent



## 配置参数


In [215]:
def get_args():
    """配置参数
    """
    curr_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    parser = argparse.ArgumentParser(description="hyperpaparameter")
    parser.add_argument("--algo_name", default="PolicyGradient", type=str, help="算法名称")
    parser.add_argument("--env_name", default="CliffWalking-v0", type=str, help="环境名称")
    parser.add_argument("--train_eps", default=400, type=int, help="训练回合数")
    parser.add_argument("--test_eps", default=20, type=int, help="测试回合数")
    parser.add_argument("--input_dim", default=4, type=int, help="状态维度")
    parser.add_argument("--hidden_dim", default=128, type=int, help="隐藏层维度")
    parser.add_argument("--n_actions", default=4, type=int, help="动作维度")
    parser.add_argument("--n_states", default=48, type=int, help="状态维度")
    parser.add_argument("--output_dim", default=2, type=int, help="动作维度")
    parser.add_argument("--gamma", default=0.9, type=float, help="折扣因子")
    parser.add_argument("--lr", default=0.001, type=float, help="学习率")
    parser.add_argument("--if_load_ckpt", default=False, type=bool, help="是否加载模型")
    parser.add_argument("--ckpt_dir", default="/kaggle/working/", type=str, help="模型存储路径")
    parser.add_argument("--device", default="cpu", type=str, help="cpu或者gpu")
    parser.add_argument("--seed", default=8, type=int, help="随机种子")
    args = parser.parse_args([])
    return args

def smooth(data, weigth=0.9):
    """用于平滑曲线，类似于Tensorboard中的smooth
    
    Args:
        data(List): 用于平滑的数组
        weigth(Float): 平滑权重，0-1之间，值越大越平滑，一般取0.9
    
    Returns:
        smoothed(List): 平滑后的数组
    """
    last = data[0] # First value in the plot (first timestep)
    smoothed = list()
    for point in data:
        smoothed_val = last * weigth + (1 - weigth) * point # 计算平滑值
        smoothed.append(smoothed_val)
        last = smoothed_val # Array is smoothed so the last value is smoothed_val
    return smoothed

def plot_rewards(rewards, cfg, tag="train"):
    sns.set_theme()
    plt.figure()
    plt.title(f"{tag}ing curve on {cfg.device} if {cfg.algo_name} for {cfg.env_name}")
    plt.plot(rewards, label="rewards")
    plt.plot(smooth(rewards), label="smoothed rewards")
    plt.legend()
    plt.show()



## 开始训练

In [216]:
class Tee:
    """同时将输出写入文件和标准输出"""
    def __init__(self, filename, mode='a'):
        self.file = open(filename, mode, encoding='utf-8')
        self.stdout = sys.stdout  # 备份原始标准输出

    def write(self, message):
        self.file.write(message)
        #self.stdout.write(message)  # 同时输出到控制台

    def flush(self):  # 必须实现flush方法
        self.file.flush()
        self.stdout.flush()

In [217]:
cfg = get_args()
cfg.ckpt_path = "ckpt/"
env, agent = env_agent_config(cfg)
sys.stdout = Tee("policygradient_log.txt", mode="w") # 写入覆盖模式
res_dic = train(cfg, env, agent)
sys.stdout = sys.stdout.stdout
plot_rewards(res_dic['rewards'], cfg, tag="train")

res_dic = test(cfg, env, agent)
plot_rewards(res_dic['rewards'], cfg, tag="test")
#agent = PGNet()

/Users/damonchang/opt/anaconda3/envs/pytorch/lib/python3.9/site-packages/gym/core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/Users/damonchang/opt/anaconda3/envs/pytorch/lib/python3.9/site-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


KeyboardInterrupt: 

In [ ]:
cfg.output_dim

4

In [ ]:
cfg

Namespace(algo_name='PolicyGradient', env_name='CliffWalking-v0', train_eps=400, test_eps=20, input_dim=18, gamma=0.9, lr=0.1, device='cpu', seed=8)